In [ ]:
ROOT_DIR = "../data/"

results = [
    ROOT_DIR + "eq/results/fine-tuned_multi-task"
]

In [ ]:
def get_values(label : str, df : pd.DataFrame) -> pd.DataFrame:
    return df[ df.iloc[:, 0] == label ]

def parse_classification_report(df : pd.DataFrame) -> dict:

    report = {}
    labels = df.iloc[:, 0].to_list()

    for label in labels:
        values = get_values(label, df)
        if label == 'accuracy':
            accuracy = values['precision'].to_list()[0]
            report[label] = accuracy
        else:
            precision, recall, f1, support = values[['precision', 'recall', 'f1-score', 'support']].iloc[0].to_list()
            report[label] = {
                'precision' : precision,
                'recall' : recall,
                'f1' : f1,
                'support' : support
            }

    return report

def collect_average_metrics(results : dict) -> dict:
    avgs = ['macro avg', 'weighted avg']
    metrics = ['precision', 'recall', 'f1']
    
    avg_metrics = {}
    avg_metrics['accuracy'] = float(np.mean([i['accuracy'] for i in results.values()]))
    
    for avg in avgs:
        for metric in metrics:

            values = [i[avg] for i in results.values()]
            values = [i[metric] for i in values]
            avg_metrics[metric + " - " + avg] = float(np.mean(values))

    return avg_metrics

In [ ]:
pd.DataFrame(evaluations).transpose()

In [ ]:
import os
import pandas as pd
import glob
import numpy as np

evaluations = {}

for result in results:

    classification_reports = glob.glob( f"{file}/evaluation_*.csv" )

    evaluation = {}
    for report in classification_reports:
        label_name = re.findall(r"evaluation_(.*).csv", report)[0]
        data = pd.read_csv(report)
        data = parse_classification_report(data)
        evaluation[label_name] = data

    avg_metrics = collect_average_metrics(evaluation)

    evaluations[result] = avg_metrics